In [ ]:
import itertools
import math
import os
import sys

import numpy as np
import pandas as pd
from numba import njit
from plotly.subplots import make_subplots
from vectorbt import FigureWidget
from vectorbt.generic import nb as generic_nb
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import vectorbt as vbt
from lib.utils import file_to_data_frame, LR, ExtendedPortfolio, plot_series_vs_scatters, dropnaninf, get_best_index, \
    where_true_set_series, shift_np, positive_return_prob, trades_count, k_mean, signals_to_ones

In [ ]:
symbol = "ADA"
candles = "_25000"
file = f"/Users/pilo/development/itba/pf/Binance_Minute_OHLC_CSVs/{candles[1:]}/Binance_{symbol}USDT_minute{candles}.csv"
_, ohlcv = file_to_data_frame(file)

fee = 0.001
min_trades = 500
save_html = True
close = ohlcv["Close"]
volume = ohlcv["Volume"]
lr_ind = LR.run(close)
print(lr_ind.lr.shape)

In [ ]:
@njit
def signal_calculations(lr, lr_ma, lr_mstd, vol, vol_mstd, lr_thld, vol_thld):
    lr_thld_std = lr_thld * lr_mstd
    ups = lr_ma - lr_thld_std # lr_thld < 0
    downs = lr_ma + lr_thld_std
    lr_entries = np.where(lr < downs, True, False)
    lr_exits = np.where(lr > ups, True, False)
    vol_thld_std = vol_thld * vol_mstd
    vol_entries = np.where(vol > vol_thld_std, True, False)
    return ups, downs, lr_entries, lr_exits, vol_thld_std, vol_entries

@njit
def ma_mstd(shifted_lr, shifted_volume, lag):
    lr_ma = generic_nb.rolling_mean_nb(shifted_lr, lag)
    lr_mstd = generic_nb.rolling_std_nb(shifted_lr, lag)
    vol_mstd = generic_nb.rolling_std_nb(shifted_volume, lag)
    return lr_ma, lr_mstd, vol_mstd

@njit
def signals_nb(lr, shifted_lr, vol, shifted_vol, lr_thld, vol_thld, lag):
    lr_ma, lr_mstd, vol_mstd = ma_mstd(shifted_lr, shifted_vol, lag)
    _, _, lr_entries, lr_exits, _, vol_entries = signal_calculations(lr, lr_ma, lr_mstd, vol, vol_mstd, lr_thld, vol_thld)
    final_entries = lr_entries & vol_entries
    return final_entries, lr_exits

In [ ]:
ENTRY_SIGNALS = vbt.IndicatorFactory(
    input_names=['lr', 'shifted_lr', 'vol', 'shifted_vol'],
    param_names=['lr_thld', 'vol_thld', 'lag'],
    output_names=['entries','exits']
).from_apply_func(signals_nb, use_ray=True)
lr_thld = -np.linspace(0,5, 30, endpoint=True)
vol_thld = np.linspace(0,10, 30, endpoint=True)
lag = list(range(6,100, 2))
signals = ENTRY_SIGNALS.run(lr=lr_ind.lr, shifted_lr=shift_np(lr_ind.lr.to_numpy(), 1),
                            vol=volume, shifted_vol=shift_np(volume.to_numpy(), 1),
                            lr_thld=lr_thld, vol_thld=vol_thld, lag=lag,
                            param_product=True, short_name="signals")
signals.entries.head()

In [ ]:
portfolio_kwargs = dict(
    direction='longonly',
    freq='m',
    size=np.inf,
    fees=fee,
    init_cash=100
)
port = ExtendedPortfolio.from_signals(close, signals.entries, signals.exits, **portfolio_kwargs, max_logs=0)

In [ ]:
port.expected_log_returns().vbt.volume().show()

In [ ]:
wr.vbt.volume().show()

In [ ]:

all_signals = signals_to_ones(signals.entries, signals.exits)

In [ ]:
vol_plots = []

In [ ]:
res_lr_avg = all_signals.vbt.reduce(k_mean, close.to_numpy(), fee, min_trades)
vol_plots.append({
    "data": dropnaninf(res_lr_avg),
    "title": "sum(log(p_exit/p_entry),..., )/ #Trades"
})
vol_plots.append({
    "data": dropnaninf(res_lr_avg[res_lr_avg>0]),
    "title": "(sum(log(p_exit/p_entry),..., )/ #Trades) > 0"
})

In [ ]:
res_prob_avg = all_signals.vbt.reduce(positive_return_prob, close.to_numpy(), fee, min_trades)
vol_plots.append({
    "data": dropnaninf(res_prob_avg),
    "title": "#Positive_trades / #Trades"
})

In [ ]:
res_trades_count = all_signals.vbt.reduce(trades_count)
vol_plots.append({
    "data": dropnaninf(res_trades_count),
    "title": "#Trades"
})

In [ ]:
res_median = all_signals.vbt.median()
res_median

In [ ]:
vol_plots.append({
    "data": dropnaninf(res_median),
    "title": "Median"
})
vol_plots.append({
    "data": dropnaninf(res_median[res_median>0]),
    "title": "Median > 0"
})

In [ ]:
for i in range(len(vol_plots)):
    vol_plots[i]["data"].vbt.volume(title=vol_plots[i]["title"]).show()

In [ ]:
best_indexes = entry_exit_prob_avg.nlargest(20, keep="all")

In [ ]:
# graficamos con el mejor de los resultados de la optimización
best_idx = best_indexes[-1]
best_lr_thld, best_vol_thld, best_lag = best_idx

In [ ]:
# recalculamos algunas cosas para graficar y entender lo que está pasando
ups, downs, lr_entries, lr_exits, vol_thld_std, vol_entries = signal_calculations(
    lr_ind.lr.to_numpy(),
    ma_mstd.lr_ma[best_lag].to_numpy(),
    ma_mstd.lr_mstd[best_lag].to_numpy(),
    volume.to_numpy(),
    ma_mstd.vol_mstd[best_lag].to_numpy(),
    best_lr_thld,
    best_vol_thld)

In [ ]:
def plot_series_vs_scatters(series_list: list, booleans_list):
    index = None
    series = series_list.pop(0)
    fig = series.vbt.plot()
    while len(series_list):
        series = series_list.pop(0)
        if not isinstance(series, pd.Series):
            series = pd.Series(series, index=index, copy=True)
        elif index is None:
            index = series.index
        series.vbt.plot(fig=fig)
    i = 1
    for scatter in booleans_list:
        if not isinstance(scatter, pd.Series):
            scatter = pd.Series(scatter, index=index, copy=True)
        elif index is None:
            index = series.index
        scatter = where_true_set_series(series, scatter)
        scatter.name = i
        i += 1
        fig = scatter.vbt.scatterplot(fig=fig)
    return fig

entries = signals.entries[best_idx]
plot_series_vs_scatters([lr_ind.lr, downs], []).show()

In [ ]:
plots = [
    plot_series_vs_scatters([lr_ind.lr, downs.T], []) #, downs, ups], [lr_entries, lr_exits]),
    #plot_series_vs_scatters([volume, vol_thld_std], [vol_entries]),
    #(entries.where(entries == True, np.nan)).vbt.scatterplot()
]

In [ ]:
def add_all_subplots(fig, row, col, list):
    for a in list:
        fig.add_trace(a, row=row, col=col)

fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                    vertical_spacing=0.02)
for i in range(len(plots)):
    add_all_subplots(fig, i+1, 1, plots[i].data)

fig.update_layout(height=700, legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()

In [ ]:
@njit
def double_multiplier_nb(values, x, y):
    return values*x, values*y

DOUBLE_MULTIPLIER = vbt.IndicatorFactory(
    input_names=['values'],
    param_names=['x', 'y'],
    output_names=['x_mu','y_mu']
).from_apply_func(double_multiplier_nb)
x = np.linspace(0,2,5, endpoint=True)
y = -np.linspace(0,2,5, endpoint=True)
tp_sl = DOUBLE_MULTIPLIER.run(mstd_ind.mstd, x, y, param_product=True, short_name="tp_sl")

In [ ]:
tp_exits = lr_ind.lr_above(tp_sl.x_mu)
sl_exits = lr_ind.lr_below(tp_sl.y_mu)
final_exits = tp_exits.vbt | sl_exits.vbt
final_exits.columns = final_exits.columns.rename("lag", level=-1)
final_exits.head()

In [ ]:
lag = 2
p = math.e ** np.array([[5],[6],[6],[3],[2],[4],[7],[8],[6],[5],[4]])
v = np.array([[5.],[6.],[6.],[3.],[2.],[4.],[7.],[8.],[6.],[5.],[4.]])
e_ln = np.log(p)
e_lr = np.array([[np.nan],[1],[0],[-3],[-1],[2],[3],[1],[-2],[-1],[-1]])
e_shifted_lr = np.array([[np.nan],[np.nan],[1],[0],[-3],[-1],[2],[3],[1],[-2], [-1]])
e_lr_ma = np.array([[np.nan], [np.nan],[np.nan],[ 1/2], [-3/2], [-2],[ 1/2],[ 5/2], [2], [-1/2], [-3/2]])
e_lr_mstd = np.array([[np.nan], [np.nan],[np.nan], [0.5], [1.5], [1. ], [1.5], [0.5], [1. ], [1.5], [0.5]])
lr = LR.run(p).lr.to_numpy()
shifted_lr = shift_np(lr, 1)
shifted_vol = shift_np(v, 1)
e_vol_mstd = np.array([[np.nan], [np.nan], [0.5], [0. ], [1.5], [0.5], [1. ], [1.5], [0.5], [1. ], [0.5]])
lr_ma, lr_mstd, vol_mstd = ma_mstd(shifted_lr, shifted_vol, 2)
assert (np.isclose(lr, e_lr, equal_nan=True).all())
assert (np.isclose(shifted_lr, e_shifted_lr, equal_nan=True).all())
assert (np.isclose(lr_ma, e_lr_ma, equal_nan=True).all())
assert (np.isclose(lr_mstd, e_lr_mstd, equal_nan=True).all())
assert (np.isclose(vol_mstd, e_vol_mstd, equal_nan=True).all())

thld = -2
e_lr_entries = e_lr < e_lr_ma + thld * e_lr_mstd
e_lr_exits = e_lr > e_lr_ma - thld * e_lr_mstd
e_vol_entries = v > -thld * vol_mstd
e_final_entries = e_lr_entries & e_vol_entries
_, _, lr_entries, lr_exits, _, vol_entries = signal_calculations(lr, lr_ma, lr_mstd, v, vol_mstd, thld, -thld)
final_entries,_ = signals_nb(lr, shifted_lr, v, shifted_vol, thld, thld, lag)
assert (np.array_equal(lr_entries, e_lr_entries))
assert (np.array_equal(lr_exits, e_lr_exits))
assert (np.array_equal(vol_entries, e_vol_entries))
assert (np.array_equal(final_entries, e_final_entries))

e_all_signals_in_ones = np.array([[np.nan],[np.nan],[np.nan],[1],[np.nan],[-1],[np.nan],[1],[1],[np.nan],[np.nan]])
all_signals_in_ones = signals_to_ones(final_entries, lr_exits, columns=[(lag, thld)])
assert (np.array_equal(all_signals_in_ones, e_all_signals_in_ones, equal_nan=True))

e_trade_count = 1
e_positive_return = 1
e_avg_lr = math.log(math.e**4/math.e**3)
avg_lr = k_mean("", all_signals_in_ones.to_numpy().flatten(), p.flatten())
positive_return = positive_return_prob("", all_signals_in_ones.to_numpy().flatten(), p.flatten(), fee)
trade_count = trades_count("", all_signals_in_ones.to_numpy().flatten())
assert (avg_lr == e_avg_lr)
assert (positive_return == e_positive_return)
assert (trade_count == e_trade_count)